# Data preparation: Embeddings and Indexing
Este notebook refere-se à etapa de preparação dos dados, contemplando o pipeline de vetorização dos dados textuais e armazenamento dos vetores semânticos para consulta. Os passos aqui apresentados são:
- Chunking dos documentos
- Vetorização (embeddings)
- Teste e indexação com FAISS
- Indexação e persistência em banco Qdrant
- Consulta de checagem da indexação

Para esta etapa, **é recomendado utilizar apenas o dataset Quati, com a tabela de `passages`**, tendo em vista que é esse conjunto de dados que contém os documentos necessários para recuperação da informação e é conjunto que define o problema de negócio proposto.

### **Considerações para vetorização e indexação**

Por ora, **foi utilizado o artefato já processado `quati_reranker_eval`**, delimitando a recuperação aos documentos que possuem consultas relevantes.

Isso se deve pelo custo computacional de vetorização dos textos exigir muito processamento. Processar todo o `quati_1M_passages`, gerou quase 2 milhões de chunks e vetorizar todos esses dados levaria mais de 4 horas em ambiente com GPU no Google Colab, o que torna inviável sem a utilização de custos.

Desta forma, é necessário a diminuição do volume de dados para produção de resultados em curto prazo.

Considerou-se aplicar um cache nas vetorizações (armazenar embeddings já processados), porém o cache somente será aplicado nos scripts de produção.

Outra alternativa futura para minização no custo computacional de armazenamento e busca de vetores focado na indexação e recuperação dos dados é a aplicação da **quantização int8** aos vetores, técnica que tem suporte nativo tanto FAISS quanto no Qdrant. Essa técnica

Caso a coleta dos dados seja feita diretamente do Hugging Face, o pré-processamento textual (limpeza e normalização) precisa ser replicado do notebook `02_preprocessing.ipynb` para padronização e consistência nas transformações aplicadas no projeto.

## Instalações e importações

In [13]:
# Check CUDA version
!nvcc --version

/bin/bash: line 1: nvcc: command not found


In [14]:
# Instalações
!pip install "datasets<4.0.0"
!pip install faiss-cpu  # faiss-gpu-cu12
!pip install qdrant-client

In [15]:
# Imports
# Utils
import os
import hashlib
import pickle
import warnings
from typing import List
from multiprocessing import cpu_count
cpu_c = cpu_count()
print(f"CPU count: {cpu_c}")
warnings.filterwarnings("ignore")

# Manipulação dos dados
import numpy as np
from datasets import load_dataset, load_from_disk

# Embeddings
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

# Indexação e persistência vetorial
import faiss
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

CPU count: 2


## Carregamento dos dados
A partir do dataset processado é preciso filtrar todos os documentos presentes para realização da vetorização e indexação.

In [16]:
# Load Quati dataset
quati_ds = load_dataset("parquet", data_files="./quati_reranker_eval_v1.parquet")["train"]

print(quati_ds)
print("-" * 100)
quati_ds[:3]

Dataset({
    features: ['query_id', 'passage_id', 'query', 'passage', 'label'],
    num_rows: 1933
})
----------------------------------------------------------------------------------------------------


{'query_id': [1, 1, 1],
 'passage_id': ['clueweb22-pt0000-78-09747_0',
  'clueweb22-pt0000-96-07278_111',
  'clueweb22-pt0001-85-06153_3'],
 'query': ['qual a maior característica da fauna brasileira?',
  'qual a maior característica da fauna brasileira?',
  'qual a maior característica da fauna brasileira?'],
 'passage': ['onça-pintada - escola kids qual matéria está procurando ? ciências home ciências onça-pintada onça-pintada que tal conhecer um pouco mais a respeito da incrível onça-pintada? neste texto falaremos sobre o maior felino encontrado nas américas. conheça as principais características desse animal, seus hábitos alimentares e sua forma de reprodução. a onça-pintada é um dos símbolos da fauna brasileira a onça-pintada é um dos símbolos da fauna do brasil e o maior felino encontrado nas américas. esse animal é um mamífero da ordem carnivora e seu nome científico é panthera onca. leia também: classificação dos mamíferos → características a onça-pintada tem como característic

In [17]:
# Get all passages from dataset
quati_ds = quati_ds.select_columns(["passage_id", "passage"])

print(quati_ds)
print("-" * 100)
quati_ds[:3]

Dataset({
    features: ['passage_id', 'passage'],
    num_rows: 1933
})
----------------------------------------------------------------------------------------------------


{'passage_id': ['clueweb22-pt0000-78-09747_0',
  'clueweb22-pt0000-96-07278_111',
  'clueweb22-pt0001-85-06153_3'],
 'passage': ['onça-pintada - escola kids qual matéria está procurando ? ciências home ciências onça-pintada onça-pintada que tal conhecer um pouco mais a respeito da incrível onça-pintada? neste texto falaremos sobre o maior felino encontrado nas américas. conheça as principais características desse animal, seus hábitos alimentares e sua forma de reprodução. a onça-pintada é um dos símbolos da fauna brasileira a onça-pintada é um dos símbolos da fauna do brasil e o maior felino encontrado nas américas. esse animal é um mamífero da ordem carnivora e seu nome científico é panthera onca. leia também: classificação dos mamíferos → características a onça-pintada tem como característica marcante a sua pelagem, que varia do amarelo-claro ao castanho com algumas manchas pretas em forma de rosetas. essas manchas funcionam como as nossas impressões digitais. assim como as impressõe

In [18]:
# Get unique passages by passage_id while retaining all columns

# Create a dictionary to store the first index for each unique passage_id
seen_passage_ids = {}
unique_indices = []

for i, item in enumerate(quati_ds):
    passage_id = item['passage_id']
    if passage_id not in seen_passage_ids:
        seen_passage_ids[passage_id] = i
        unique_indices.append(i)

# Filter the dataset to keep only the rows with unique passage_ids
quati_ds = quati_ds.select(unique_indices)

print(quati_ds)
print("-" * 100)
print(quati_ds[:3])

Dataset({
    features: ['passage_id', 'passage'],
    num_rows: 1896
})
----------------------------------------------------------------------------------------------------
{'passage_id': ['clueweb22-pt0000-78-09747_0', 'clueweb22-pt0000-96-07278_111', 'clueweb22-pt0001-85-06153_3'], 'passage': ['onça-pintada - escola kids qual matéria está procurando ? ciências home ciências onça-pintada onça-pintada que tal conhecer um pouco mais a respeito da incrível onça-pintada? neste texto falaremos sobre o maior felino encontrado nas américas. conheça as principais características desse animal, seus hábitos alimentares e sua forma de reprodução. a onça-pintada é um dos símbolos da fauna brasileira a onça-pintada é um dos símbolos da fauna do brasil e o maior felino encontrado nas américas. esse animal é um mamífero da ordem carnivora e seu nome científico é panthera onca. leia também: classificação dos mamíferos → características a onça-pintada tem como característica marcante a sua pelagem, q

## Chunking
Para realização do chunking dos documentos, é obrigatoriamente necessário **utilizar o mesmo modelo de vetorização dos textos na tokenização** (útil para o chunking de dados por tokens, que mantém a semântica no trecho). No tópico de vetorização, tem o detalhamento do modelo selecionado.

Os chunks foram delimitados com até **256 tokens** com **overlap de 64 tokens** entre os chunks.

Esse modelo é relativamente leve e pouco custoso, além de possuir semântica multilinguística e ser focado para busca semântica.


In [19]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "intfloat/multilingual-e5-small",
    # device="cuda"
)

print(f"Model selected: {tokenizer.name_or_path}")
print(f"Tokenizer max length: {tokenizer.model_max_length}")
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")

Model selected: intfloat/multilingual-e5-small
Tokenizer max length: 512
Tokenizer vocab size: 250002


In [20]:
# Function to chunk text
def chunk_text(text: str, max_tokens: int = 256, overlap: int = 64) -> List[str]:
    """
    Divide um texto em trechos (chunks) a partir de tokens, mantendo um overlap entre os trechos.
    Cada trecho é representado por uma lista de tokens contendo até max_tokens palavras.
    O overlap entre os trechos é definido por overlap tokens.

    Params:
        text (str): Texto a ser dividido em trechos.
        max_tokens (int): Número máximo de tokens por trecho.
        overlap (int): Número de tokens de overlap entre os trechos.

    Returns:
        List[str]: Lista de trechos do texto.
    """
    # Create tokens
    tokens = tokenizer(
        text,
        truncation=False,
        return_offsets_mapping=False,
        add_special_tokens=False,
    )["input_ids"]

    # Set chunks
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        # Set chunk ids
        chunk_ids = tokens[i:i + max_tokens]
        if not chunk_ids:
            continue

        # Get chunks
        chunk = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append(chunk)
    return chunks


# Function alternative: uses offsets mapping to split the chunks by tokens
def chunk_by_tokens(text, chunk_size=384, overlap=64):
    tokens = tokenizer(
        text,
        return_offsets_mapping=True,
        add_special_tokens=False,
    )
    input_ids = tokens["input_ids"]
    offsets = tokens["offset_mapping"]

    chunks = []
    start = 0

    while start < len(input_ids):
        end = start + chunk_size
        chunk_offsets = offsets[start:end]
        char_start = chunk_offsets[0][0]
        char_end = chunk_offsets[-1][1]
        chunks.append(text[char_start:char_end])
        start += chunk_size - overlap

    return chunks

In [21]:
# Applying chunks into documents
def apply_chunks(batch):
    batch["chunks"] = [chunk_text(doc) for doc in batch["passage"]]
    return batch

quati_ds = quati_ds.map(
    apply_chunks,
    batched=True,
    batch_size=10_000,
    num_proc=cpu_c
)

quati_ds.select_columns(["chunks", "passage"])[:2]

{'chunks': [['onça-pintada - escola kids qual matéria está procurando? ciências home ciências onça-pintada onça-pintada que tal conhecer um pouco mais a respeito da incrível onça-pintada? neste texto falaremos sobre o maior felino encontrado nas américas. conheça as principais características desse animal, seus hábitos alimentares e sua forma de reprodução. a onça-pintada é um dos símbolos da fauna brasileira a onça-pintada é um dos símbolos da fauna do brasil e o maior felino encontrado nas américas. esse animal é um mamífero da ordem carnivora e seu nome científico é panthera onca. leia também: classificação dos mamíferos → características a onça-pintada tem como característica marcante a sua pelagem, que varia do amarelo-claro ao castanho com algumas manchas pretas em forma de rosetas. essas manchas funcionam como as nossas impressões digitais. assim como as impressões em nossos dedos, as manchas da onça-pintada são únicas. as onças-pintadas apresentam peso que varia de 35 kg a 130 

In [22]:
# Flatten the chunks for indexing
texts = []
metadatas = []

for row in quati_ds:
    for idx, chunk in enumerate(row["chunks"]):
        texts.append(chunk)
        metadatas.append({
            "passage_id": row["passage_id"],
            "chunk_id": idx
        })

print(f"Total of chunks: {len(texts)}")

Total of chunks: 3615


## Vetorização dos dados (embeddings)
Na seleção do modelo para vetorização dos textos, foram buscado modelos recentes no Hugging Face que apresentem boas perfomances e precisão, com semântica multilinguística e, principalmente, com baixa latência e alta velocidade em inferência e no uso.

A partir de uma breve pesquisa de modelos para embedding, o modelo que melhor atende as especificações necessárias foi o modelo `Multilingual-E5-Small`.

Dentre os modelos considerados, mantendo o foco em velocidade e baixo custo, abaixo segue a descrição das possibilidades avaliadas:

| **Modelo**                     | **Descrição**                                                                                                      | **Latência/Velocidade**                                        | **Memória**                              | **Recursos/Observações**                                                                 |
|---------------------------------|---------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------|------------------------------------------|-------------------------------------------------------------------------------------------|
| **Multilingual-E5-Small**       | Modelo "all-rounder" otimizado para buscas semânticas em mais de 100 idiomas.                                       | "Campeão de velocidade", latência de 16ms, até 7x mais rápido que modelos baseados em LLMs. | Menos de 200MB de RAM                   | Ideal para RAGs em produção que exigem alta taxa de transferência (QPS).                 |
| **EmbeddingGemma-300M**        | Lançado em setembro de 2025 pelo Google, baseado na arquitetura Gemma 3.                                             | Sub-segundo, projetado para execução local e on-device.         | Requer apenas 300MB de RAM (cai para 100-200MB com quantização). | Suporta Matryoshka (MRL), permitindo reduzir vetores de 768 para 128 dimensões sem grande perda de precisão. |
| **BGE-Small-v1.5**             | Versão compacta da família BAAI, focada em tarefas de recuperação técnica e RAG.                                     | Latência competitiva abaixo de 30ms.                           | Semelhante ao E5-Small, utiliza dimensões reduzidas (384). | Melhor para quem precisa de estabilidade na distribuição de similaridade (evita scores inflados). |
| **MPNet-Base-v2 (Multilingual)** | Modelo clássico para detecção de paráfrases e similaridade curta.                                                   | Rápido (~25ms), mas superado pelos modelos acima em precisão de busca. | ~420MB                                  | Limitado a contextos curtos (128 tokens), dificultando uso em RAGs com parágrafos longos. |


In [23]:
# Initialize model
model = SentenceTransformer(
    "intfloat/multilingual-e5-small",
    # device="cuda"
)

print(f"Model max sequence length: {model.max_seq_length}")
print(f"Model embedding dimension: {model.get_sentence_embedding_dimension()}")

Model max sequence length: 512
Model embedding dimension: 384


In [24]:
# Create embeddings
embeddings = model.encode(
    texts,
    show_progress_bar=True,
    batch_size=16,
    normalize_embeddings=True   # For equalizing search methods
)

print(f"Shape of embeddings: {embeddings.shape}")
print(f"Quantidade de vetores: {embeddings.shape[0]}")
print(f"Quantidade de dimensões: {embeddings.shape[1]}")

Batches:   0%|          | 0/226 [00:00<?, ?it/s]

Shape of embeddings: (3615, 384)
Quantidade de vetores: 3615
Quantidade de dimensões: 384


### Cache de embeddings
Para otimização do processo de vetorização dos dados, a aplicação de uma técnica de cache dos embeddings gerados se torna pré-requisito para base de dados muito grandes (Big Data).

O cache de embeddings se torna bastante prático para não ter retrabalho em processar embeddings que já foram gerados uma vez.

A ideia central do cache é verificar se o dado a ser trabalhado já não foi processado. Ou seja:
```
texto → hash → existe?
        ├── sim → carrega embedding
        └── não → encode() → salva → retorna
```
Esse processo pode ser aplicado através de um Vector DB também, contanto que respeite esse fluxo: verifica se o texto (ou textos) já foi transformado (através de um identificador único ou hash); se sim, recarrega o embedding para o texto; se não, gera o novo embedding para o dado.

>TODO: Aplicar cache com uso de Qdrant.

Existem duas formas para realizar o cache:
- Guardar individualmente os embeddings em cache (exige mais memória de armazenamento, verificar linear de cada embedding).
- Guardar em batches os embeddings no cache (maior complexidade, útil para paralelismo).

In [ ]:
# Folder to save cache locally
CACHE_DIR = "data/embeddings_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

print(f"Cache folder: {CACHE_DIR}")

Cache folder: data/embeddings_cache


In [26]:
# Function to hash the text
# Useful for saving and getting embeddings already processed
def text_hash(text: str) -> str:
    return hashlib.md5(text.encode("utf-8")).hexdigest()

#### Cache individual de embeddings

In [27]:
# Define functions to save, load and verify embeddings
# Function to cache embedding
def save_embedding(hash_id, embedding):
    with open(f"{CACHE_DIR}/{hash_id}.pkl", "wb") as f:
        pickle.dump(embedding, f)

# Function to get cached embedding
def load_embedding(hash_id):
    with open(f"{CACHE_DIR}/{hash_id}.pkl", "rb") as f:
        return pickle.load(f)

def is_cached(text: str) -> bool:
    return os.path.exists(f"{CACHE_DIR}/{text_hash(text)}.pkl")

In [ ]:
# Function to get or compute embeddings
def get_or_compute_embedding(text: str, model: SentenceTransformer) -> np.ndarray:
    """
    Busca ou computa a vetorização para um determinado texto.
    A função primeiro verifica se determinado texto já foi vetorizado através do hash do seu texto.
    Caso não tenha sido vetorizado, aplica o encoding do modelo ao texto.
    TODO: Aprimorar cache com Qdrant.

    Params:
        text (str): Texto a ser vetorizado.
        model (SentenceTransformer): Modelo de vetorização.

    Returns:
        np.ndarray: Vetorização do texto.

    """
    h = text_hash(text)
    
    # Verify if text is already vectorized
    if is_cached(text):
        return load_embedding(h)

    # Vectorize text
    embedding = model.encode(
        text,
        normalize_embeddings=True
    )

    # Save embedding
    save_embedding(text_hash(text), embedding)

    return embedding

In [29]:
# Run embeddings with cache
# embeddings = np.vstack([
#     get_or_compute_embedding(text, model)
#     for text in texts
# ])

#### Cache em lotes de embeddings (TODO)

## Indexação e validação local com FAISS
A critério de aprendizado, a indexação feita nesse momento utilizará a biblioteca `FAISS` que permite buscas de similaridades rápidas através da crição de estruturas de dados (índices) que organizam os vetores de dados.

Importante destacar que o **FAISS é nativamente compatível com bibliotecas como Langchain e LlamaIndex para utilização em pipelines RAG**.

O **método de indexação utilizado foi o `IndexFlat`** que permite armazenamento dos vetores sem compressão e busca 100% exata comparando a consulta com cada vetor.

Além desse método, é importante considerar o método `Inverted File Index` (IVF) que separa o espaço vetorial em clusters com k-means, e método HNSW baseado em grafo multicamadas.

In [30]:
# Create index
dim = embeddings.shape[1]   # Vector dimension for embeddings
index = faiss.IndexFlatIP(dim)  # Index Flat method uses Brute force search

In [31]:
# Transfer index to GPU (index 0 for GPU)
# gpu_res = faiss.StandardGpuResources()  # Create GPU resource

# index = faiss.index_cpu_to_gpu(gpu_res, 0, index)

In [32]:
# Add vectors (embeddings) into index
index.add(embeddings)

print(f"Total of indexed vectors: {index.ntotal}")

Total of indexed vectors: 3615


In [33]:
# Quick search (top 5 chunks)
k = 5
distances, indices = index.search(np.array([embeddings[0]]), k)

# Display
print(f"Corresponding text: {texts[0]}")
print(f"Corresponding chunk: {metadatas[0]}")
print("=" * 100)

for i in range(len(indices[0])):
    print(f"Distance: {distances[0][i]}")
    print(f"Text: {texts[indices[0][i]]}")
    print(f"Metadata: {metadatas[indices[0][i]]}")
    print("-" * 100)

Corresponding text: onça-pintada - escola kids qual matéria está procurando? ciências home ciências onça-pintada onça-pintada que tal conhecer um pouco mais a respeito da incrível onça-pintada? neste texto falaremos sobre o maior felino encontrado nas américas. conheça as principais características desse animal, seus hábitos alimentares e sua forma de reprodução. a onça-pintada é um dos símbolos da fauna brasileira a onça-pintada é um dos símbolos da fauna do brasil e o maior felino encontrado nas américas. esse animal é um mamífero da ordem carnivora e seu nome científico é panthera onca. leia também: classificação dos mamíferos → características a onça-pintada tem como característica marcante a sua pelagem, que varia do amarelo-claro ao castanho com algumas manchas pretas em forma de rosetas. essas manchas funcionam como as nossas impressões digitais. assim como as impressões em nossos dedos, as manchas da onça-pintada são únicas. as onças-pintadas apresentam peso que varia de 35 kg 

In [ ]:
# Persist index and embeddings

# Move the index back to CPU before saving
# index = faiss.index_gpu_to_cpu(index)
faiss.write_index(index, "faiss_index.faiss")   # Index
np.save("embeddings.npy", embeddings)   # Embeddings
print("Index saved to: faiss_index.faiss")
print("Embeddings saved to: embeddings.npy")

Index saved to: faiss_index.faiss
Embeddings saved to: embeddings.npy


## Persistência dos dados com Qdrant
Para persistência real dos embeddings e utilização de um Vector DB dentro da aplicabilidade do projeto, optamos por utilizar o Qdrant.

**Qdrant é um banco de dados vetorial (Vector Database) e motor de busca por similaridade de código aberto**, projetado especificamente para lidar com buscas semânticas e aplicações de IA em larga escala.

O Qdrant utiliza o **algoritmo HNSW para busca semântica**, realiza segmentação de dados e permite o **armazenamento de metadados `payload` para filtragem híbrida**, além de armazenamento flexível tanto em RAM quanto em disco.

In [35]:
# Initialize DB
client = QdrantClient(path="./ragbr_qdrant_db")

In [36]:
# Create colletion for chunks
client.recreate_collection(
    collection_name="quati_chunks",
    vectors_config=VectorParams(
        size=dim,   # Size = Embeddings dimension
        distance=Distance.COSINE    # Method to calculate distance
    )
)

True

In [37]:
# Upsert data
points = [
    PointStruct(
        id=idx,
        vector=embeddings[idx].tolist(),
        payload={
            "text": texts[idx],
            "metadata": metadatas[idx]
        }
    )
    for idx in range(len(texts))
]

client.upsert(
    collection_name="quati_chunks",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

## Query check
Abaixo, segue uma checagem rápida da busca vetorial através do Qdrant.

In [ ]:
# Define a query
query = "O que é o Imposto de Renda para Pessoa Física?"    # Not cleaned
query_emb = model.encode(query, normalize_embeddings=True)

print(f"Query: {query}")
print(f"Query embedding shape: {query_emb.shape}")

Query: O que é o Imposto de Renda para Pessoa Física?
Query embedding shape: (384,)


In [39]:
# Search for chunks with similarity
hits = client.query_points(
    collection_name="quati_chunks",
    query=query_emb.tolist(),
    limit=k
).points

for h in hits:
    print(f"Score: {h.score}")
    print(f"Text: {h.payload['text']}")
    print(f"Metadata: {h.payload['metadata']}")
    print("-" * 100)

Score: 0.8781166662362213
Text: de fluxo de caixa, por exemplo. porém a tributação do imposto de renda é de 22,5% sobre os rendimentos. tesouro selic essa modalidade possui renda fixa e rendimentos diários atrelados ao sistema especial de liquidação e custódia (selic), conhecida como taxa básica de juros.
Metadata: {'passage_id': 'clueweb22-pt0002-07-07946_7', 'chunk_id': 1}
----------------------------------------------------------------------------------------------------
Score: 0.8761338051225848
Text: impostos nas operações do tesouro direto são o imposto sobre operações financeiras (iof), para resgates da aplicação em menos de 30 dias, e o imposto de renda (ir), com alíquota regressiva a depender do prazo do investimento. quanto custa investir no tesouro direto? existem diversas taxas do tesouro direto que os investidores precisam conhecer antes de aplicar o dinheiro nesse tipo de investimento.
Metadata: {'passage_id': 'clueweb22-pt0000-20-20743_4', 'chunk_id': 1}
----------------

In [40]:
# Zipp qdrant database for exporting
!zip -r ragbr_qdrant_db.zip ragbr_qdrant_db

  adding: ragbr_qdrant_db/ (stored 0%)
  adding: ragbr_qdrant_db/.lock (stored 0%)
  adding: ragbr_qdrant_db/collection/ (stored 0%)
  adding: ragbr_qdrant_db/collection/quati_chunks/ (stored 0%)
  adding: ragbr_qdrant_db/collection/quati_chunks/storage.sqlite (deflated 57%)
  adding: ragbr_qdrant_db/meta.json (deflated 56%)
